In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
import re
# from gensim.models import Word2Vec
# import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale
import numpy as np
import pandas as pd
import os, sys, getopt, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
import itertools
from textblob import TextBlob
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
C:\Users\konark\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\konark\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\konark\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\konark\anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
data = pd.read_csv(r"C:\Users\konark\Downloads\Hindi-English\Hindi_english.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()
data['Text'] = data['Text'].str.replace('[^\w\s]','')

In [3]:
#####STOPWORDS############
stopwords = pd.read_csv(r"C:\Users\konark\Downloads\Hindi-English\stopwords.csv")
stopwords = stopwords['Vocabulary'].tolist()

In [4]:
data['Text'] = data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [5]:
vocabulary = pd.read_csv(r"C:\Users\konark\Downloads\Hindi-English\Hindi_Eng_vocab.csv")
vocabulary['Vocabulary'] = vocabulary['Vocabulary'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Vocabulary'], vocabulary['Generalised Spelling']))

In [6]:
data["Text"] = data["Text"].str.replace('[^\w\s]','')

In [7]:
# d2 = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}

import pandas as pd
import nltk

# df = pd.DataFrame({'sentences': ['This is a very good site. I will recommend it to others.', 'Can you please give me a call at 9983938428. have issues with the listings.', 'good work! keep it up']})
data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)

In [8]:
# replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
# data['Text'] = data['Text'].replace(d2, regex=True)
for sub in data['tokenized_sents']:
    for i, word in enumerate(sub):
        if word in voc_to_be_used:
            sub[i] = voc_to_be_used[word]

In [9]:
# data.head()
toekn_to_sentence = []
# for token in list(data['tokenized_sents']):
#     toekn_to_sentence.append([' '.join(i) for i in token])
toekn_to_sentence.append([[' '.join(i)] for i in list(data['tokenized_sents'])])

In [10]:

final_list = []
for i in range(len(toekn_to_sentence[0])):
    final_list.append(toekn_to_sentence[0][i][0])


In [11]:
# final_list

In [12]:
data['Token_to_sentence'] = final_list

In [13]:
data.head()

,Text,Sentiment,tokenized_sents,Token_to_sentence
0,coach dismissed nhi big jeetenge nhi,0,"[coach, dismiss, nhi, big, jeetenge, nhi]",coach dismiss nhi big jeetenge nhi
1,kewal bhagao team jeetne dhanyawad,0,"[kewal, bhag, team, jeetna, dhanyavad]",kewal bhag team jeetna dhanyavad
2,run media baate media villen esliye kamini med...,0,"[run, media, baat, media, villain, isliye, kam...",run media baat media villain isliye kamine med...
3,devs innings won masterpiece,2,"[devs, innings, won, masterpiece]",devs innings won masterpiece
4,bhot nice lot respect heart,2,"[bahut, nice, lot, respect, heart]",bahut nice lot respect heart


In [14]:
data.drop(['Text', 'tokenized_sents'], axis = 1, inplace=True)
data.rename(columns={'Token_to_sentence': 'Text'}, inplace=True)

In [15]:
# data.head()

In [16]:
import itertools
for i in range(len(data)):
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
data.head()

,Sentiment,Text
0,0,coach dismiss nhi big jeetenge nhi
1,0,kewal bhag team jeetna dhanyavad
2,0,run media baat media villain isliye kamine med...
3,2,devs innings won masterpiece
4,2,bahut nice lot respect heart


In [18]:
# Preprocessing
def tokens(message):
#     message = unicode(message, 'utf8')
    return TextBlob(message).words

def lemmas(message):
#     message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [72]:
# Divide data into Test and Train
msg_train, msg_test, label_train, label_test = train_test_split(data['Text'], data['Sentiment'], test_size=0.15, random_state=42)

In [73]:
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [74]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[205  30  41]
 [ 49  74  70]
 [ 47  25 206]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.74      0.71       276
           1       0.57      0.38      0.46       193
           2       0.65      0.74      0.69       278

    accuracy                           0.65       747
   macro avg       0.63      0.62      0.62       747
weighted avg       0.64      0.65      0.64       747



In [30]:
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [31]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[239  13  18]
 [ 75  33  37]
 [ 35   9 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       270
           1       0.60      0.23      0.33       145
           2       0.78      0.82      0.80       238

    accuracy                           0.71       653
   macro avg       0.69      0.64      0.63       653
weighted avg       0.70      0.71      0.68       653



In [32]:
# Uni-Bi
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])
# pipeline parameters to automatically explore and tune
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__alpha': [1, 1e-1, 1e-2, 1e-3]
}
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
nb_detector = grid.fit(msg_train, label_train)

In [33]:
print (confusion_matrix(label_test, nb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, nb_detector.predict(msg_test)))

[[239  13  18]
 [ 75  33  37]
 [ 35   9 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.89      0.77       270
           1       0.60      0.23      0.33       145
           2       0.78      0.82      0.80       238

    accuracy                           0.71       653
   macro avg       0.69      0.64      0.63       653
weighted avg       0.70      0.71      0.68       653



In [75]:
###############################################SUPPORT VECTOR MACHINES##########################################################
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

In [76]:
print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[191  55  30]
 [ 37 115  41]
 [ 42  59 177]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.69      0.70       276
           1       0.50      0.60      0.55       193
           2       0.71      0.64      0.67       278

    accuracy                           0.65       747
   macro avg       0.64      0.64      0.64       747
weighted avg       0.66      0.65      0.65       747



In [36]:
# Uni-bi
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[226  22  22]
 [ 59  52  34]
 [ 29  20 189]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       270
           1       0.55      0.36      0.44       145
           2       0.77      0.79      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [37]:
# Uni
from sklearn.svm import SVC, LinearSVC
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', SVC())])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = [
        {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
        {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf', 'linear']},
]
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
svc_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, svc_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, svc_detector.predict(msg_test)))

[[226  22  22]
 [ 59  52  34]
 [ 29  20 189]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.84      0.77       270
           1       0.55      0.36      0.44       145
           2       0.77      0.79      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [77]:
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

In [78]:
print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[200  50  26]
 [ 36 110  47]
 [ 41  54 183]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.72      0.72       276
           1       0.51      0.57      0.54       193
           2       0.71      0.66      0.69       278

    accuracy                           0.66       747
   macro avg       0.65      0.65      0.65       747
weighted avg       0.67      0.66      0.66       747



In [40]:
#Uni-Bi
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[227  21  22]
 [ 50  52  43]
 [ 24  20 194]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.75      0.84      0.80       270
           1       0.56      0.36      0.44       145
           2       0.75      0.82      0.78       238

    accuracy                           0.72       653
   macro avg       0.69      0.67      0.67       653
weighted avg       0.71      0.72      0.71       653



In [41]:
#Uni
#############SGC CLassifier##################################
from sklearn.linear_model import SGDClassifier
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', SGDClassifier(max_iter=1000))])
# pipeline parameters to automatically explore and tune
# params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
params = {
'tfidf__use_idf': (True, False),
'bow__analyzer': (lemmas, tokens),
'tfidf__norm': ('l1', 'l2'),
'classifier__loss': ["hinge", "log", "squared_hinge", "modified_huber"],
"classifier__alpha" : [0.0001, 0.001, 0.01, 0.1],
'classifier__penalty' : ["l2", "l1", "none"]
}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
sgd_detector = grid.fit(msg_train, label_train)

print (confusion_matrix(label_test, sgd_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, sgd_detector.predict(msg_test)))

[[226  21  23]
 [ 56  48  41]
 [ 29  16 193]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.73      0.84      0.78       270
           1       0.56      0.33      0.42       145
           2       0.75      0.81      0.78       238

    accuracy                           0.72       653
   macro avg       0.68      0.66      0.66       653
weighted avg       0.70      0.72      0.70       653



In [79]:
##########################LSTM IMPLEMENTATION##############################
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
#LSTM Implementation
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 14000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 6498 unique tokens.


In [80]:
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4979, 250)


In [81]:
data.head()

,Sentiment,Text
0,0,coach dismiss nhi big jeetenge nhi
1,0,kewal bhag team jeetna dhanyavad
2,0,run media baat media villain isliye kamine med...
3,2,devs innings won masterpiece
4,2,bahut nice lot respect heart


In [82]:
Y = pd.get_dummies(data['Sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (4979, 3)


In [83]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4481, 250) (4481, 3)
(498, 250) (498, 3)


In [84]:
# variables_for_classification=3 #change it as per your number of categories
model = Sequential()
# model.add(Dense(variables_for_classification, activation='softmax'))
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          1400000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                1313      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 42        
Total params: 1,481,755
Trainable params: 1,481,755
Non-trainable params: 0
_________________________________________________________________
None


In [85]:
epochs = 100
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 4032 samples, validate on 449 samples
Epoch 1/100
4032/4032 [==============================] - 192s 48ms/step - loss: 1.0932 - accuracy: 0.3743 - val_loss: 1.0686 - val_accuracy: 0.4410
Epoch 2/100
4032/4032 [==============================] - 188s 47ms/step - loss: 1.0161 - accuracy: 0.5119 - val_loss: 0.9720 - val_accuracy: 0.5390
Epoch 3/100
4032/4032 [==============================] - 183s 45ms/step - loss: 0.9075 - accuracy: 0.6248 - val_loss: 0.9435 - val_accuracy: 0.6125
Epoch 4/100
4032/4032 [==============================] - 40s 10ms/step - loss: 0.8032 - accuracy: 0.7393 - val_loss: 0.9013 - val_accuracy: 0.6370
Epoch 5/100
4032/4032 [==============================] - 38s 9ms/step - loss: 0.7046 - accuracy: 0.8073 - val_loss: 0.9098 - val_accuracy: 0.6169
Epoch 6/100
4032/4032 [==============================] - 36s 9ms/step - loss: 0.6208 - accuracy: 0.8522 - val_loss: 0.9001 - val_accuracy: 0.6169
Epoch 7/100
4032/4032 

In [86]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} %'.format(accr[0],accr[1]*100))

498/498 [==============================] - 3s 6ms/step
Test set
  Loss: 1.744
  Accuracy: 58.835 %


In [87]:
y_pred = model.predict(X_test, verbose=0)

In [88]:
import sklearn.metrics as metrics
matrix = metrics.confusion_matrix(Y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [89]:
print (confusion_matrix(Y_test.argmax(axis=1), y_pred.argmax(axis=1)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(Y_test.argmax(axis=1), y_pred.argmax(axis=1)))

[[123  42  36]
 [ 19  68  32]
 [ 34  42 102]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.70      0.61      0.65       201
           1       0.45      0.57      0.50       119
           2       0.60      0.57      0.59       178

    accuracy                           0.59       498
   macro avg       0.58      0.59      0.58       498
weighted avg       0.60      0.59      0.59       498



In [90]:
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[160  83  33]
 [ 33 122  38]
 [ 30  93 155]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.72      0.58      0.64       276
           1       0.41      0.63      0.50       193
           2       0.69      0.56      0.62       278

    accuracy                           0.59       747
   macro avg       0.60      0.59      0.58       747
weighted avg       0.63      0.59      0.59       747



In [63]:
###################Uni-Bi#######################
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,2))),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[209  27  34]
 [ 56  46  43]
 [ 31  19 188]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       270
           1       0.50      0.32      0.39       145
           2       0.71      0.79      0.75       238

    accuracy                           0.68       653
   macro avg       0.64      0.63      0.62       653
weighted avg       0.66      0.68      0.66       653



In [64]:
from xgboost import XGBClassifier
import xgboost
######XGBoost Implementation####################
#############Uni######
# create pipeline
pipeline = Pipeline([('bow', CountVectorizer(analyzer=lemmas)),('tfidf', TfidfTransformer()),('classifier', XGBClassifier())])

params = {'tfidf__use_idf': (True, False),'bow__analyzer': (lemmas, tokens),'tfidf__norm': ('l1', 'l2'),'classifier__min_child_weight': [5, 10], 'classifier__gamma': [2, 5], 'classifier__subsample': [0.6], 'classifier__max_depth': [5,10]}

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(msg_train, label_train)
grid = GridSearchCV(
    pipeline,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)
# train
xgb_detector = grid.fit(msg_train, label_train)

import numpy as np
# temsg_testdiction_list = np.array(xgb_detector.predict(msg_test))
print (confusion_matrix(label_test, xgb_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, xgb_detector.predict(msg_test)))

[[209  27  34]
 [ 56  46  43]
 [ 31  19 188]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       270
           1       0.50      0.32      0.39       145
           2       0.71      0.79      0.75       238

    accuracy                           0.68       653
   macro avg       0.64      0.63      0.62       653
weighted avg       0.66      0.68      0.66       653



In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,3))
data_text = vectorizer.fit_transform(data['Text'])
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(data_text, data['Sentiment'], test_size=0.2, random_state=42)
###SVM##########################
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

# pipe1 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', MultinomialNB())])
# pipe2 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', SVC())])
# pipe3 = Pipeline([('bow', CountVectorizer(analyzer=lemmas, ngram_range = (1,3))),('classifier', SGDClassifier(max_iter=1000))])

mnb = MultinomialNB()
svc = SVC(probability=True)
sgd = SGDClassifier(max_iter=1000)
lr = LogisticRegression(multi_class='multinomial', solver='newton-cg')
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier()
voting_clf = VotingClassifier(estimators=[('SVC', svc),
                                          ('MNB', mnb),
                                          ('LR', lr),
                                          ('SGD', sgd)],
#                                           ('GBM', gbm)],
                                          voting='soft')

params = {'SVC__C': [1, 10, 100], 'SVC__gamma': [0.001], 'SVC__kernel': ['rbf', 'linear'],
          'MNB__alpha': [1, 1e-1, 1e-2],
          'LR__penalty' : ['l2'],'LR__C' : np.logspace(-4, 4, 4),
          'SGD__loss': ["log", "modified_huber"],"SGD__alpha" : [0.001, 0.01],
          'SGD__penalty' : ["l2", "l1"]}
#           'GBM__loss':["deviance"], 'GBM__max_features': ['auto', 'sqrt', 'log2'], 'GBM__max_depth':[3,5,8], 'GBM__max_features':["log2","sqrt"]}                                          
skf = StratifiedKFold(n_splits=3)
skf.get_n_splits(msg_train, label_train)

grid = GridSearchCV(
    voting_clf,
    params, # parameters to tune via cross validation
    refit=True, # fit using all data, on the best detected classifier
    n_jobs=-1,
    scoring='accuracy',
    cv=skf,
)

ensembling_detector = grid.fit(msg_train, label_train)
print (confusion_matrix(label_test, ensembling_detector.predict(msg_test)))
print ("")
print (":: Classification Report")
print ("")
print (classification_report(label_test, ensembling_detector.predict(msg_test)))

[[309  27  21]
 [ 75  77  38]
 [ 40  29 254]]

:: Classification Report

              precision    recall  f1-score   support

           0       0.73      0.87      0.79       357
           1       0.58      0.41      0.48       190
           2       0.81      0.79      0.80       323

    accuracy                           0.74       870
   macro avg       0.71      0.69      0.69       870
weighted avg       0.73      0.74      0.73       870



In [70]:

import joblib
filename = r'C:\Users\konark\Downloads\finalized_model_v4.sav'
joblib.dump(ensembling_detector, filename)                                          

['C:\\Users\\konark\\Downloads\\finalized_model_v4.sav']

In [145]:
data = pd.read_csv(r"C:\Users\konark\Downloads\punjabi_data_v2.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()

vocabulary = pd.read_csv(r"C:\Users\konark\Downloads\generalised_spellings.csv")
vocabulary['Words'] = vocabulary['Words'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Words'], vocabulary['Generalised Spelling']))

replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
data['Text'] = data['Text'].replace(replacement_dict, regex=True)

stopwords = ["100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
# stopwords = ["lokk",'abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["lokk","abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", "10" ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["kasoor", "ssp", "prateek","tu", "wabba", "pyn", "ah","mnu","mai", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
import itertools
for i in range(len(data)):
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
########LSTM + WORD2VEC+TF-IDF##################

def cleaning_text(original_series):
    original_series = original_series.str.replace('(ap)', '')
    converting_nt = original_series.str.replace(r'n\'t',' not')
    
    removing_trailing_spaces_lowering = converting_nt.str.strip().str.lower()
    
    removing_special_characters = removing_trailing_spaces_lowering.str.replace(r'[!.:,#-$\\\'0-9]','')
    removing_bracket_starting = removing_special_characters.str.replace(r'\(','')
    removing_bracket_ending = removing_bracket_starting.str.replace(r'\)','')
    splitting_into_words = removing_bracket_ending.str.split()
    words_greater_than_zero = splitting_into_words[splitting_into_words.map(lambda d: len(d)) > 0]
    return (words_greater_than_zero)

clean_sentences = cleaning_text(data['Text'])

In [92]:
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data.Sentiment, test_size=.2)

In [93]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
LabeledSentence = gensim.models.doc2vec.LabeledSentence

def labelizeStories(stories, label_type):
    labelized = []
    for i,v in enumerate(stories):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

X_train = labelizeStories(X_train, 'TRAIN')
X_test = labelizeStories(X_test, 'TEST')

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if __name__ == '__main__':


In [94]:
X_train[0]

LabeledSentence(words='nhi', tags=['TRAIN_0'])

In [95]:
from gensim.models import Word2Vec

model_w2v = Word2Vec(list(clean_sentences), min_count=2,sg=1,window=8)

In [96]:
print (model_w2v)

Word2Vec(vocab=2705, size=100, alpha=0.025)


In [97]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 4210 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

Loading BokehJS ...

In [98]:
word_vectors = [model_w2v[w] for w in model_w2v.wv.vocab.keys()]

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [99]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = model_w2v.wv.vocab.keys()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2705 samples in 0.505s...
[t-SNE] Computed neighbors for 2705 samples in 2.935s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2705
[t-SNE] Computed conditional probabilities for sample 2000 / 2705
[t-SNE] Computed conditional probabilities for sample 2705 / 2705
[t-SNE] Mean sigma: 0.012451
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.484528
[t-SNE] KL divergence after 1000 iterations: 0.791110


In [100]:
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

In [101]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=2)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 43


In [102]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
#     count = 0.
    count = 1
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size)) * tfidf[word]
#             count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [103]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x.words, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x.words, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [104]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=100))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [105]:
model.fit(train_vecs_w2v, y_train, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
 - 3s - loss: -7.0999e-01 - accuracy: 0.3103
Epoch 2/100
 - 1s - loss: -1.1583e+00 - accuracy: 0.3103
Epoch 3/100
 - 1s - loss: -1.1748e+00 - accuracy: 0.3103
Epoch 4/100
 - 1s - loss: -1.1812e+00 - accuracy: 0.3103
Epoch 5/100
 - 1s - loss: -1.1876e+00 - accuracy: 0.3103
Epoch 6/100
 - 1s - loss: -1.1905e+00 - accuracy: 0.3103
Epoch 7/100
 - 1s - loss: -1.1931e+00 - accuracy: 0.3103
Epoch 8/100
 - 1s - loss: -1.1989e+00 - accuracy: 0.3103
Epoch 9/100
 - 1s - loss: -1.2024e+00 - accuracy: 0.3103
Epoch 10/100
 - 1s - loss: -1.1974e+00 - accuracy: 0.3103
Epoch 11/100
 - 1s - loss: -1.2089e+00 - accuracy: 0.3103
Epoch 12/100
 - 1s - loss: -1.2015e+00 - accuracy: 0.3103
Epoch 13/100
 - 1s - loss: -1.2061e+00 - accuracy: 0.3103
Epoch 14/100
 - 2s - loss: -1.2117e+00 - accuracy: 0.3103
Epoch 15/100
 - 1s - loss: -1.2117e+00 - accuracy: 0.3103
Epoch 16/100
 - 1s - loss: -1.2177e+00 - accuracy: 0.3103
Epoch 17/100
 - 1s - loss: -1.2146e+00 - accuracy: 0.3103
Epoch 18/100
 - 1s - lo

In [106]:
score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)

In [107]:
print ('Accuracy', score[1]*100,'%')

Accuracy 28.21285128593445 %


In [20]:
###############################BI-DIRECTIONAL LSTM##########################################
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import glob

In [21]:
# Define some Global Variables
max_features = 100000 # Maximum Number of words we want to include in our dictionary
maxlen = 72 # No of words in question we want to create a sequence with
embed_size = 300# Size of word to vec embedding we are using

In [14]:
data = pd.read_csv(r"C:\Users\konark\Downloads\punjabi_data_v2.csv", encoding='latin-1', engine='python' )
data['Text'] = data['Text'].str.lower()

vocabulary = pd.read_csv(r"C:\Users\konark\Downloads\generalised_spellings.csv")
vocabulary['Words'] = vocabulary['Words'].str.lower()
vocabulary['Generalised Spelling'] = vocabulary['Generalised Spelling'].str.lower()
voc_to_be_used = dict(zip(vocabulary['Words'], vocabulary['Generalised Spelling']))

replacement_dict = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k,v in voc_to_be_used.items()}
data['Text'] = data['Text'].replace(replacement_dict, regex=True)

stopwords = ["100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
# stopwords = ["lokk",'abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["lokk","abp","video", "aafull", "me", "rhe", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", "10" ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae", "bs", "chal", "jagdeep", "captian", "anchor", "ranjeet", "amrinder", "ikk", "ina", "dilli", "mai", "te", "fr","india", "singa", "tn", "dharti", "gurdas", "singer", "sound", "live", "respiratory", "punjab", "punjabian", "jd", "chd", "geet", "ta", "bt", "koe", "nusrat", "fateh", "sudhu", "sidhu", "sukhbir", "interview", "tumhe", "agr", "eh", "uncle", "but", "covid", "dong", "jidaan", "matlab", "tere", "abc", "ik", "for","news", "sanja", "tuc", "tum", "detail", "account", "accounts", "interview", "or", "diya", "add", "addmission", "address", "ade", "admin", "advocate"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
stopwords = ["k","c","kasoor", "ssp", "prateek","tu", "wabba", "pyn", "ah","mnu","mai", "100ch", "march", "april", "21", "2018", "april", "20", "1st", "19", "15", "urs", "14", "11", "108", "100", '10' ,"ch", "di", "aa","wich","bai","meri","v","isa","jisa","vica","taka","vi","othon","nahim","bhi","valom","iha ","iha ","jadom","kai","tada","andar","utte","sabuta","kadi","nem","ji","kise","pūra","ne","hove","jekar","de","jehara","baada","sara","cho","kade","sab","tan","ki","na","huna","jinam","nala","cahe","kisa ","pichom","edhara","nu","ajihe","hi","ke","hain","bahuta","kaafi","huṇe","lai","ki","magara","da","tarham","phera","vele","othe","kite","ithe","jinhanu","jad","vanga ","doraan","varaga","jo","la","pura","naale","ton","hona","paso","jeha","es","jina","kujh","dobara","sadda","ethe","bare","kad","kadde","hoye","rahe","bano","deṇi","pia ","hoia","gai","laga","huda","janda","vēkha","suṇa","ai","sakde","jave","janda","karke","bilkul","eho","kaun","pher","tad","kolon","kina","jive","hethan","sare","jithe","koi","ki","je","dian","chala","lai","aakh","baṇa","kara","pain","keh","chuke","keha", "karvayei","banaye","kitta","javan","dekh","adi","lia","karana","lagoda","aave","kari","laeya","reh","uha","sam","sabha","hana","tu","si","ho", "tennu","tusa","hain","hai","apna","je","aate","jam","kal","vagaira","rakh","laag","gal","pi","a","reha","geya","otha","rahi","usne","tusi","mera","usdi","tera","us","oye","aap","san","mein","tusi","assi","par","te","tam","bhavem","aagali","varg","ama","la","hala","ek", "tuhada","kita","karde","krde","teri","apne","aj","singh","sikh","ohne","dekh","tuhadi","fir","time","bawa","sidhu","vich","sach","naal","hor","ki","mainu","aaa","wale","hor","krn","kudi","kuj", "12", "2020", "30", "300", "pwa", "do", "log", "3may", "35", "40", "din", "400", "50", "km", "door", "5911", "50m", "3bisby", "500", "6month", "700", "900", "__", "honi", "aab", "which", "ohi", "hr", "to", "aae"]
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
data['Text'] = data['Text'].str.replace(pat, '')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
import itertools
for i in range(len(data)):
    data['Text'][i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(data['Text'][i]))

C:\Users\konark\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
tokenizer = Tokenizer(num_words=20000, split=' ')

tokenizer.fit_on_texts(data['Text'])
X = tokenizer.texts_to_sequences(data['Text'])
X = pad_sequences(X)
# Y = data['Sentiment']

Y = pd.get_dummies(data['Sentiment']).values
print('Shape of label tensor:', Y.shape)

# We can then create our train and test sets:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)

# X_train, X_test, y_train, y_test = train_test_split(data['Text'], data.Sentiment, test_size=.15)

Shape of label tensor: (4979, 3)


In [23]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4232, 59) (4232, 3)
(747, 59) (747, 3)


In [24]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

In [25]:
from keras.preprocessing import sequence
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(Y_train)
y_test = np.array(Y_test)

Pad sequences (samples x time)
x_train shape: (4232, 100)
x_test shape: (747, 100)


In [26]:
# model = Sequential()
# model.add(Embedding(11000, embed_dim = 128, input_length = X.shape[1], dropout=0.2))
# model.add( Bidirectional( LSTM(lstm_out = 196, dropout_U = 0.2, dropout_W = 0.2)))
# model.add( Dense(3, activation = 'softmax'))

# model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [27]:
#  try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [28]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
#           validation_data=[x_test, y_test]
         )

Train...
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
4232/4232 [==============================] - 29s 7ms/step - loss: 0.6088 - accuracy: 0.6836
Epoch 2/100
4232/4232 [==============================] - 29s 7ms/step - loss: 0.4574 - accuracy: 0.7831
Epoch 3/100
4232/4232 [==============================] - 30s 7ms/step - loss: 0.3344 - accuracy: 0.8583
Epoch 4/100
4232/4232 [==============================] - 37s 9ms/step - loss: 0.2302 - accuracy: 0.9147
Epoch 5/100
4232/4232 [==============================] - 41s 10ms/step - loss: 0.1743 - accuracy: 0.9356
Epoch 6/100
4232/4232 [==============================] - 38s 9ms/step - loss: 0.1435 - accuracy: 0.9460
Epoch 7/100
4232/4232 [==============================] - 35s 8ms/step - loss: 0.1217 - accuracy: 0.9557
Epoch 8/100
4232/4232 [==============================] - 36s 9ms/step - loss: 0.1081 - accuracy: 0.9607
Epoch 9/100
4232/4232 [==============================] - 29s 7ms/step - loss: 0.1031 - accuracy: 0.9612
Epoch 

In [29]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} %'.format(accr[0],accr[1]*100))

747/747 [==============================] - 1s 2ms/step
Test set
  Loss: 2.017
  Accuracy: 73.315 %


In [30]:
y_pred = model.predict(x_test, verbose=0)

In [31]:
np.round(y_pred[0],0).astype(int)

array([1, 0, 0])

In [32]:
y_test[0]

array([1, 0, 0], dtype=uint8)

In [33]:
y_pred_max = [np.round(p,0).astype(int) for p in y_pred]

In [34]:
y_pred_max[0]

array([1, 0, 0])

In [35]:
print (":: Classification Report")
print ("")
print (classification_report(y_test, y_pred_max))

:: Classification Report

              precision    recall  f1-score   support

           0       0.68      0.62      0.65       276
           1       0.49      0.58      0.53       193
           2       0.62      0.58      0.60       278

   micro avg       0.60      0.60      0.60       747
   macro avg       0.60      0.59      0.59       747
weighted avg       0.61      0.60      0.60       747
 samples avg       0.59      0.60      0.59       747



C:\Users\konark\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
